In [1]:
using_colab = 'google.colab' in str(get_ipython())
print("Using Colab:", using_colab)
if using_colab:
    !pip install -U "xformers==0.0.33" --index-url https://download.pytorch.org/whl/cu126
    !pip install diffusers


Using Colab: False


In [2]:
import torch
from data.spair import SPairDataset
from torch.utils.data import DataLoader
import os
from pathlib import Path

from utils.utils_featuremaps import PreComputedFeaturemaps

base_dir = os.path.abspath(os.path.curdir)

if using_colab:
    base_dir = os.path.join(os.path.abspath(os.path.curdir), 'AML-polito')


def collate_single(batch_list):
    return batch_list[0]

save_dir = Path(base_dir) / "data" / "features"
dataset_size = 'large'  # 'small' or 'large'

# Load dataset and construct dataloader

test_dataset = SPairDataset(datatype='test', dataset_size=dataset_size)

test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1, collate_fn=collate_single)
print("Dataset loaded")

Dataset loaded


In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
torch.backends.cuda.matmul.allow_tf32 = True



Device: cuda


In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from typing import Any, Callable, Dict, List, Optional, Union, Tuple
from diffusers.models.unets.unet_2d_condition import UNet2DConditionModel
from diffusers import DDIMScheduler
import gc
import os


class MyUNet2DConditionModel(UNet2DConditionModel):
    def forward(
            self,
            sample: torch.Tensor,
            timestep: Union[torch.Tensor, float, int],
            encoder_hidden_states: torch.Tensor,
            up_ft_indices,
            class_labels: Optional[torch.Tensor] = None,
            timestep_cond: Optional[torch.Tensor] = None,
            attention_mask: Optional[torch.Tensor] = None,
            cross_attention_kwargs: Optional[Dict[str, Any]] = None,
            added_cond_kwargs: Optional[Dict[str, torch.Tensor]] = None,
            down_block_additional_residuals: Optional[Tuple[torch.Tensor]] = None,
            mid_block_additional_residual: Optional[torch.Tensor] = None,
            down_intrablock_additional_residuals: Optional[Tuple[torch.Tensor]] = None,
            encoder_attention_mask: Optional[torch.Tensor] = None,
            return_dict: bool = True
    ):
        r"""
        Args:
            sample (`torch.FloatTensor`): (batch, channel, height, width) noisy inputs tensor
            timestep (`torch.FloatTensor` or `float` or `int`): (batch) timesteps
            encoder_hidden_states (`torch.FloatTensor`): (batch, sequence_length, feature_dim) encoder hidden states
            cross_attention_kwargs (`dict`, *optional*):
                A kwargs dictionary that if specified is passed along to the `AttnProcessor` as defined under
                `self.processor` in
                [diffusers.cross_attention](https://github.com/huggingface/diffusers/blob/main/src/diffusers/models/cross_attention.py).
        """
        # By default samples have to be AT least a multiple of the overall upsampling factor.
        # The overall upsampling factor is equal to 2 ** (# num of upsampling layears).
        # However, the upsampling interpolation output size can be forced to fit any upsampling size
        # on the fly if necessary.
        default_overall_up_factor = 2 ** self.num_upsamplers

        # upsample size should be forwarded when sample is not a multiple of `default_overall_up_factor`
        forward_upsample_size = False
        upsample_size = None

        if any(s % default_overall_up_factor != 0 for s in sample.shape[-2:]):
            # logger.info("Forward upsample size to force interpolation output size.")
            forward_upsample_size = True

        # prepare attention_mask
        if attention_mask is not None:
            attention_mask = (1 - attention_mask.to(sample.dtype)) * -10000.0
            attention_mask = attention_mask.unsqueeze(1)

        # 0. center input if necessary
        if self.config.center_input_sample:
            sample = 2 * sample - 1.0

        # 1. time
        timesteps = timestep
        if not torch.is_tensor(timesteps):
            # TODO: this requires sync between CPU and GPU. So try to pass timesteps as tensors if you can
            # This would be a good case for the `match` statement (Python 3.10+)
            is_mps = sample.device.type == "mps"
            if isinstance(timestep, float):
                dtype = torch.float32 if is_mps else torch.float64
            else:
                dtype = torch.int32 if is_mps else torch.int64
            timesteps = torch.tensor([timesteps], dtype=dtype, device=sample.device)
        elif len(timesteps.shape) == 0:
            timesteps = timesteps[None].to(sample.device)

        # broadcast to batch dimension in a way that's compatible with ONNX/Core ML
        timesteps = timesteps.expand(sample.shape[0])

        t_emb = self.time_proj(timesteps)

        # timesteps does not contain any weights and will always return f32 tensors
        # but time_embedding might actually be running in fp16. so we need to cast here.
        # there might be better ways to encapsulate this.
        t_emb = t_emb.to(dtype=self.dtype)

        emb = self.time_embedding(t_emb, timestep_cond)

        if self.class_embedding is not None:
            if class_labels is None:
                raise ValueError("class_labels should be provided when num_class_embeds > 0")

            if self.config.class_embed_type == "timestep":
                class_labels = self.time_proj(class_labels)

            class_emb = self.class_embedding(class_labels).to(dtype=self.dtype)
            emb = emb + class_emb

        # 2. pre-process
        sample = self.conv_in(sample)

        # 3. down
        down_block_res_samples = (sample,)
        for downsample_block in self.down_blocks:
            if hasattr(downsample_block, "has_cross_attention") and downsample_block.has_cross_attention:
                sample, res_samples = downsample_block(
                    hidden_states=sample,
                    temb=emb,
                    encoder_hidden_states=encoder_hidden_states,
                    attention_mask=attention_mask,
                    cross_attention_kwargs=cross_attention_kwargs,
                )
            else:
                sample, res_samples = downsample_block(hidden_states=sample, temb=emb)

            down_block_res_samples += res_samples

        # 4. mid
        if self.mid_block is not None:
            sample = self.mid_block(
                sample,
                emb,
                encoder_hidden_states=encoder_hidden_states,
                attention_mask=attention_mask,
                cross_attention_kwargs=cross_attention_kwargs,
            )

        # 5. up
        up_ft = {}
        for i, upsample_block in enumerate(self.up_blocks):

            if i > np.max(up_ft_indices):
                break

            is_final_block = i == len(self.up_blocks) - 1

            res_samples = down_block_res_samples[-len(upsample_block.resnets):]
            down_block_res_samples = down_block_res_samples[: -len(upsample_block.resnets)]

            # if we have not reached the final block and need to forward the
            # upsample size, we do it here
            if not is_final_block and forward_upsample_size:
                upsample_size = down_block_res_samples[-1].shape[2:]

            if hasattr(upsample_block, "has_cross_attention") and upsample_block.has_cross_attention:
                sample = upsample_block(
                    hidden_states=sample,
                    temb=emb,
                    res_hidden_states_tuple=res_samples,
                    encoder_hidden_states=encoder_hidden_states,
                    cross_attention_kwargs=cross_attention_kwargs,
                    upsample_size=upsample_size,
                    attention_mask=attention_mask,
                )
            else:
                sample = upsample_block(
                    hidden_states=sample, temb=emb, res_hidden_states_tuple=res_samples, upsample_size=upsample_size
                )

            if i in up_ft_indices:
                up_ft[i] = sample.detach()

        output = {}
        output['up_ft'] = up_ft
        return output


class OneStepSDPipeline(StableDiffusionPipeline):
    @torch.no_grad()
    def __call__(
            self,
            img_tensor,
            t,
            up_ft_indices,
            prompt_embeds: Optional[torch.FloatTensor] = None,
    ):
        device = self._execution_device
        latents = self.vae.encode(img_tensor).latent_dist.sample() * self.vae.config.scaling_factor
        t = torch.tensor(t, dtype=torch.long, device=device)
        noise = torch.randn_like(latents).to(device)
        latents_noisy = self.scheduler.add_noise(latents, noise, t)
        unet_output = self.unet(
            latents_noisy,
            t,
            encoder_hidden_states=prompt_embeds,
            up_ft_indices=up_ft_indices,
        )
        return unet_output

In [ ]:
def encode_prompt_embeds(pipe, prompt: str, device: str = "cuda"):
    # Tokenizza anche la stringa vuota e produce embeddings validi [1, 77, dim]
    text_inputs = pipe.tokenizer(
        [prompt],
        padding="max_length",
        max_length=pipe.tokenizer.model_max_length,
        truncation=True,
        return_tensors="pt",
    )
    input_ids = text_inputs.input_ids.to(device)
    attention_mask = getattr(text_inputs, "attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(device)

    with torch.no_grad():
        out = pipe.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        prompt_embeds = out[0]
    return prompt_embeds

In [ ]:
import torch

class SDFeaturizer4Eval():
    def __init__(self, sd_id='Manojb/stable-diffusion-2-1-base', null_prompt='', cat_list=[]):
        unet = MyUNet2DConditionModel.from_pretrained(sd_id, subfolder="unet")
        onestep_pipe = OneStepSDPipeline.from_pretrained(sd_id, unet=unet, safety_checker=None)
        onestep_pipe.vae.decoder = None
        onestep_pipe.scheduler = DDIMScheduler.from_pretrained(sd_id, subfolder="scheduler")
        onestep_pipe = onestep_pipe.to("cuda")
        onestep_pipe.enable_attention_slicing()
        onestep_pipe.enable_xformers_memory_efficient_attention()
        null_prompt_embeds = encode_prompt_embeds(onestep_pipe, null_prompt, device="cuda")  # [1, 77, dim]

        self.null_prompt_embeds = null_prompt_embeds
        self.null_prompt = null_prompt
        self.pipe = onestep_pipe
        with torch.no_grad():
            cat2prompt_embeds = {}
            print("start encoding prompts for categories...")
            for cat in cat_list:
                print("encoding prompt for category:", cat)
                prompt = f"a photo of a {cat}"
                prompt_embeds = encode_prompt_embeds(self.pipe, prompt, device="cuda")  # [1, 77, dim]
                cat2prompt_embeds[cat] = prompt_embeds
            print("encoded prompts for categories:", list(cat2prompt_embeds.keys()))
            self.cat2prompt_embeds = cat2prompt_embeds

        self.pipe.tokenizer = None
        self.pipe.text_encoder = None
        torch.cuda.empty_cache()

    @torch.no_grad()
    def forward(self,
                img_tensor,
                category=None,
                t=261,
                up_ft_index=1,
                ensemble_size=8):
        '''
        Args:
            img_tensor: should be a single torch tensor in the shape of [1, C, H, W] or [C, H, W]
            prompt: the prompt to use, a string
            t: the time step to use, should be an int in the range of [0, 1000]
            up_ft_index: which upsampling block of the U-Net to extract feature, you can choose [0, 1, 2, 3]
            ensemble_size: the number of repeated images used in the batch to extract features
        Return:
            unet_ft: a torch tensor in the shape of [1, c, h, w]
        '''

        if img_tensor is not None:
            img_tensor = img_tensor.cuda()  # ensem, c, h, w
        if category in self.cat2prompt_embeds:
            prompt_embeds = self.cat2prompt_embeds[category]
        else:
            prompt_embeds = self.null_prompt_embeds
        prompt_embeds = prompt_embeds.repeat(ensemble_size, 1, 1).cuda()
        unet_ft_all = self.pipe(
            img_tensor=img_tensor,
            t=t,
            up_ft_indices=[up_ft_index],
            prompt_embeds=prompt_embeds)
        unet_ft = unet_ft_all['up_ft'][up_ft_index]  # ensem, c, h, w
        unet_ft = unet_ft.mean(0, keepdim=True)  # 1,c,h,w
        return unet_ft

In [ ]:
import torch
from torchvision import transforms

def resize_image_sd(
    sample: torch.Tensor,
    img_output_size: tuple[int, int] = (768, 768),  # (H, W)
    ensemble_size: int = 8,
) -> torch.Tensor:
    """
    sample: (C,H,W) con valori 0..255 (uint8 o float)
    return: (E,C,H',W') normalizzata in [-1,1]
    """
    if sample.ndim != 3:
        raise ValueError(f"`sample` deve essere (C,H,W). Trovato shape={tuple(sample.shape)}")

    resize = transforms.Resize(
        img_output_size,
        interpolation=transforms.InterpolationMode.BICUBIC,
        antialias=True
    )

    img = resize(sample)                      # (C,H',W')
    img = img.to(torch.float32)
    img = (img / 255.0 - 0.5) * 2.0          # [-1,1]
    img = img.unsqueeze(0).repeat(ensemble_size, 1, 1, 1)  # (E,C,H',W')
    return img




In [ ]:
from pathlib import Path
from tqdm import tqdm

spair71k_categories = [
    "aeroplane",
    "bicycle",
    "bird",
    "boat",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "dog",
    "horse",
    "motorbike",
    "person",
    "pottedplant",
    "sheep",
    "train",
    "tvmonitor",
]
save_dir = Path(base_dir) / "data" / "features"
print("Saving features to:", save_dir)

enseble_size = 2
torch.cuda.empty_cache()

with torch.no_grad():
    dift = SDFeaturizer4Eval(cat_list=spair71k_categories)
    with PreComputedFeaturemaps(save_dir, device=device) as pcm:
        for img_tensor, img_size, img_category, img_name in tqdm(
                test_dataset.iter_images(),
                total=test_dataset.num_images(),
                desc="Generating embeddings"
        ):
            img_tensor = img_tensor.to(device)  # [1,3,H,W]
            img_transformed = resize_image_sd(img_tensor, ensemble_size=enseble_size)

            featmap = dift.forward(img_tensor=img_transformed, category=img_category, ensemble_size=enseble_size)

            pcm.save_featuremaps(featmap, img_category, img_name)

In [7]:
def kp_src_to_featmap(
    kp_orig: torch.Tensor,
    orig_hw: tuple[int, int],          # (H, W) originali
    feat_hw: tuple[int, int] = (48, 48),
    pre_hw: tuple[int, int] = (768, 768),
) -> torch.Tensor:
    """
    Converte keypoint originali (pixel) -> indici sulla feature map (x_idx, y_idx).

    Args:
        kp_orig: (N,2) keypoint in pixel sull'immagine originale, ordine (x,y)
        orig_hw: (H,W) dell'immagine originale
        feat_hw: (hv,wv) della feature map (es. 48x48)
        pre_hw:  (Hpre,Wpre) della preprocessata (es. 768x768)

    Returns:
        (N,2) indici featuremap in ordine (x_idx, y_idx)
    """
    if kp_orig.ndim != 2 or kp_orig.shape[-1] != 2:
        raise ValueError(f"`kp_orig` deve essere (N,2). Trovato shape={tuple(kp_orig.shape)}")

    H, W = orig_hw
    hv, wv = feat_hw
    Hpre, Wpre = pre_hw

    kp = kp_orig.to(torch.float32)

    # -------------------------
    # FASE 1) RESIZE (orig -> preprocess)
    # -------------------------
    sx_img = Wpre / W
    sy_img = Hpre / H

    # mapping coerente coi centri pixel: (x+0.5)*s - 0.5
    x_pre = (kp[:, 0] + 0.5) * sx_img - 0.5
    y_pre = (kp[:, 1] + 0.5) * sy_img - 0.5

    # -------------------------
    # FASE 2) TRASLAZIONE su FEATURE MAP (preprocess -> feat idx)
    # -------------------------
    sx_feat = Wpre / wv  # stride in preprocess (es. 16)
    sy_feat = Hpre / hv  # stride in preprocess (es. 16)

    # "center rule": token center a (i+0.5)*stride  => i ≈ x/stride - 0.5
    x_idx = torch.round(x_pre / sx_feat - 0.5).long()
    y_idx = torch.round(y_pre / sy_feat - 0.5).long()

    # clamp ai limiti della feature map
    x_idx = x_idx.clamp(0, wv - 1)
    y_idx = y_idx.clamp(0, hv - 1)

    return torch.stack([x_idx, y_idx], dim=1)  # (N,2) (x_idx,y_idx)



def feat_idx_to_orig_xy(x_idx: int, y_idx: int,
                        orig_hw: tuple[int,int],
                        feat_hw=(48,48), pre_hw=(768,768)) -> tuple[float,float]:
    H, W = orig_hw
    hv, wv = feat_hw
    Hpre, Wpre = pre_hw
    sx = Wpre / wv  # 16
    sy = Hpre / hv  # 16

    # centro cella in preprocess
    x_pre = (x_idx + 0.5) * sx
    y_pre = (y_idx + 0.5) * sy

    # preprocess -> orig (dato il resize forzato)
    x = x_pre * (W / Wpre)
    y = y_pre * (H / Hpre)
    return x, y

In [15]:
import math
from itertools import islice
import torch.nn as nn
from utils.utils_correspondence import hard_argmax
from utils.utils_results import CorrespondenceResult
from tqdm import tqdm

max_images = 10
data = islice(test_dataloader, max_images)
size = max_images

results = []
torch.cuda.empty_cache()

with torch.no_grad():
    with PreComputedFeaturemaps(save_dir, device=device) as pcm:
        for batch in tqdm(data, total=size, desc=f"Elaborazione con DIFT"):
            category = batch["category"]

            orig_size_src = tuple(batch["src_imsize"][-2:])  # (H, W)
            orig_size_trg = tuple(batch["trg_imsize"][-2:])  # (H, W)

            src_imname = batch["src_imname"]
            trg_imname = batch["trg_imname"]

            pth = torch.load(save_dir / f"{category}.pth", map_location="cpu")
            #src_ft = pcm.load_featuremaps(category, src_imname)  # [C,hs,ws]
            #trg_ft = pcm.load_featuremaps(category, trg_imname)  # [C,ht,wt]

            src_ft = pth[src_imname].to("cpu")  # [C,hs,ws]
            trg_ft = pth[trg_imname].to("cpu")  # [C

            # Keypoints & metadata
            src_kps = batch["src_kps"].to("cpu")  # [N,2]
            trg_kps = batch["trg_kps"].to("cpu")
            # [N,2]
            pck_thr_0_05 = batch["pck_threshold_0_05"]
            pck_thr_0_1 = batch["pck_threshold_0_1"]
            pck_thr_0_2 = batch["pck_threshold_0_2"]

            #src_ft = nn.Upsample(size=orig_size_src, mode='bilinear')(src_ft)
            #trg_ft = nn.Upsample(size=orig_size_trg, mode='bilinear')(trg_ft)

            src_kps = kp_src_to_featmap(src_kps, orig_size_src)
            #trg_kps = kp_src_to_featmap(trg_kps, orig_size_trg)

            distances_this_image = []

            N_kps = src_kps.shape[0]
            C = src_ft.shape[1]                 # 1280
            hv_t, wv_t = trg_ft.shape[-2:]      # 48,48

            for i in range(N_kps):
                src_kp = src_kps[i]   # (x,y) originali
                trg_kp = trg_kps[i]   # (x,y) originali

                # se nel dataset possono esserci kp non validi, controlla qui
                if torch.isnan(src_kp).any() or torch.isnan(trg_kp).any():
                    continue

                # 1) indice sulla featuremap (48x48) per il kp sorgente
                x_src_idx = int(src_kp[0].item())
                y_src_idx = int(src_kp[1].item())

                # 2) vettore feature sorgente (C,1,1) per broadcasting
                src_vec = src_ft[0, :, y_src_idx, x_src_idx].view(C, 1, 1)

                # 3) mappa similarità su target (48,48)
                sim2d = torch.cosine_similarity(trg_ft[0], src_vec, dim=0, eps=1e-8)

                # 4) argmax sulla featuremap
                x_idx_t, y_idx_t = hard_argmax(sim2d)

                # 5) rimappa a pixel originali target
                x_pred, y_pred = feat_idx_to_orig_xy(x_idx_t, y_idx_t, orig_size_trg)

                # distanza rispetto al GT (trg_point è (x,y))
                dist = math.sqrt((x_pred - trg_kp[0]) ** 2 + (y_pred - trg_kp[1]) ** 2)
                distances_this_image.append(dist)

                del src_vec

            results.append(
                CorrespondenceResult(
                    category=category,
                    distances=distances_this_image,
                    pck_threshold_0_05=pck_thr_0_05,
                    pck_threshold_0_1=pck_thr_0_1,
                    pck_threshold_0_2=pck_thr_0_2
                )
            )

Elaborazione con DIFT: 100%|██████████| 10/10 [00:04<00:00,  2.06it/s]


In [16]:
from utils.utils_results import compute_pckt_images, compute_correct_per_category, compute_pckt_keypoints
# Compute and print results
correct = compute_correct_per_category(results)
compute_pckt_keypoints(correct)
compute_pckt_images(correct)

PCK Results per keypoints (%):
    Category  PCK 0.05    PCK 0.1   PCK 0.2
0  aeroplane  39.56044  56.043956  70.32967
1        All  39.56044  56.043956  70.32967
PCK per-image (%):
    Category   PCK 0.05    PCK 0.1    PCK 0.2
0  aeroplane  37.617133  56.656177  71.323427
1        All  37.617133  56.656177  71.323427
